# Getting Started with Valor Core

## Introduction

Valor is a centralized evaluation store which makes it easy to measure, explore, and rank model performance. Valor empowers data scientists and engineers to evaluate the performance of their machine learning pipelines and use those evaluations to make better modeling decisions in the future. For a conceptual introduction to Valor, [check out our project overview](https://striveworks.github.io/valor/).

In this notebook, we'll introduce Valor's high-level abstractions and walk through a computer vision-oriented example of how you can use Valor to evaluate model performance. For task-specific examples, please see our follow-up notebooks below:

- [Tabular classification](https://github.com/Striveworks/valor/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/valor/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/valor/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)

Note that this notebook uses `valor_core`, rather than `valor`, to calculate all metrics locally without utilizing Postgres' filtering and data exploration capabilities.

## Defining Our Dataset


To begin, we import all needed packages from `valor_core`. For instructions on setting up your environment, please see [our docs here](https://striveworks.github.io/valor/getting_started/).


In [1]:
from pathlib import Path

from valor_core import (
    Datum,
    Annotation,
    GroundTruth,
    Prediction,
    Label,
    Box,
    evaluate_classification, 
    evaluate_detection
)

### Creating Image Classification GroundTruths and Predictions


To describe the various objects in our data, we'll create lists of `GroundTruth` and `Prediction` objects to pass into our `evaluate..` functions. Note that Valor doesn't actually store any images, and that the `Annotations` we use will vary by our task type (i.e., object detection, semantic segmentation, etc.). For demonstrative purposes, we'll create `GroundTruths` for two different learning tasks in this notebook.


In [2]:
def create_image_classification_data(classification_data):

    groundtruths, predictions = [], []

    for element in classification_data:

        datum = Datum(
            uid=Path(element["path"]).stem, metadata={"path": element["path"]}
        )

        gt_annotations = [
            Annotation(
                labels=[
                    Label(key=key, value=value)
                    for label in element["gt_annotations"]
                    for key, value in label.items()
                ]
            )
        ]

        pd_annotations = [
            Annotation(
                labels=[
                    Label(
                        key="class_label",
                        value=label["class_label"],
                        score=label["score"],
                    )
                    for label in element["pd_annotations"]
                ]
            )
        ]

        groundtruths.append(
            GroundTruth(
                datum=datum,
                annotations=gt_annotations,
            )
        )

        predictions.append(
            Prediction(
                datum=datum,
                annotations=pd_annotations,
            )
        )

    return groundtruths, predictions


classification_data = [
        {
            "path": "a/b/c/img1.png",
            "gt_annotations": [{"class_label": "dog"}],
            "pd_annotations": [
                {"class_label": "dog", "score": 0.9},
                {"class_label": "cat", "score": 0.1},
            ],
        },
        {
            "path": "a/b/c/img2.png",
            "gt_annotations": [{"class_label": "cat"}],
            "pd_annotations": [
                {"class_label": "dog", "score": 0.1},
                {"class_label": "cat", "score": 0.9},
            ],
        },
    ]


classification_gts, classification_pds = create_image_classification_data(classification_data)
print(classification_gts)

[GroundTruth(datum=Datum(uid='img1', metadata={'path': 'a/b/c/img1.png'}), annotations=[Annotation(labels=[Label(key='class_label', value='dog', score=None)], metadata=None, bounding_box=None, polygon=None, raster=None, embedding=None, is_instance=None, implied_task_types=None)]), GroundTruth(datum=Datum(uid='img2', metadata={'path': 'a/b/c/img2.png'}), annotations=[Annotation(labels=[Label(key='class_label', value='cat', score=None)], metadata=None, bounding_box=None, polygon=None, raster=None, embedding=None, is_instance=None, implied_task_types=None)])]


### Creating Object Detection GroundTruths and Predictions


In [3]:
def create_groundtruth_from_object_detection_dict(detection_data):
    groundtruths, predictions = [], []

    for element in detection_data:

        datum = Datum(
            uid=Path(element["path"]).stem, metadata={"path": element["path"]}
        )

        gt_annotations = [
            Annotation(
                labels=[
                    Label(key="class_label", value=annotation["class_label"])
                ],
                bounding_box=Box.from_extrema(
                    xmin=annotation["bbox"]["xmin"],
                    xmax=annotation["bbox"]["xmax"],
                    ymin=annotation["bbox"]["ymin"],
                    ymax=annotation["bbox"]["ymax"],
                ),
                is_instance=True,
            )
            for annotation in element["gt_annotations"]
            if len(annotation) > 0
        ]

        pd_annotations = [
            Annotation(
                labels=[
                    Label(
                        key="class_label",
                        value=label["class_label"],
                        score=label["score"],
                    )
                    for label in annotation["labels"]
                ],
                bounding_box=Box.from_extrema(
                    xmin=annotation["bbox"]["xmin"],
                    xmax=annotation["bbox"]["xmax"],
                    ymin=annotation["bbox"]["ymin"],
                    ymax=annotation["bbox"]["ymax"],
                ),
                is_instance=True,
            )
            for annotation in element["pd_annotations"]
            if len(annotation) > 0
        ]

        groundtruths.append(
            GroundTruth(
                datum=datum,
                annotations=gt_annotations,
            )
        )

        predictions.append(
            Prediction(
                datum=datum,
                annotations=pd_annotations,
            )
        )

    return groundtruths, predictions


detection_data = [
    {
        "path": "a/b/c/img3.png",
        "gt_annotations": [
            {
                "class_label": "dog",
                "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150},
            },
            {
                "class_label": "person",
                "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110},
            },
        ],
        "pd_annotations": [
            {
                "labels": [
                    {"class_label": "dog", "score": 0.8},
                    {"class_label": "cat", "score": 0.1},
                    {"class_label": "person", "score": 0.1},
                ],
                "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150},
            },
            {
                "labels": [
                    {"class_label": "dog", "score": 0.05},
                    {"class_label": "cat", "score": 0.05},
                    {"class_label": "person", "score": 0.9},
                ],
                "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110},
            },
        ],
    },
    {
        "path": "a/b/c/img4.png",
        "gt_annotations": [
            {
                "class_label": "cat",
                "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260},
            }
        ],
        "pd_annotations": [
            {
                "labels": [
                    {"class_label": "dog", "score": 0.8},
                    {"class_label": "cat", "score": 0.1},
                    {"class_label": "person", "score": 0.1},
                ],
                "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260},
            }
        ],
    },
    {"path": "a/b/c/img5.png", "gt_annotations": [], "pd_annotations": []},
]


detection_gts, detection_pds = create_groundtruth_from_object_detection_dict(
    detection_data=detection_data
)
print(detection_pds)

[Prediction(datum=Datum(uid='img3', metadata={'path': 'a/b/c/img3.png'}), annotations=[Annotation(labels=[Label(key='class_label', value='dog', score=0.8), Label(key='class_label', value='cat', score=0.1), Label(key='class_label', value='person', score=0.1)], metadata=None, bounding_box=Box(value=[[(16, 130), (70, 130), (70, 150), (16, 150), (16, 130)]]), polygon=None, raster=None, embedding=None, is_instance=True, implied_task_types=None), Annotation(labels=[Label(key='class_label', value='dog', score=0.05), Label(key='class_label', value='cat', score=0.05), Label(key='class_label', value='person', score=0.9)], metadata=None, bounding_box=Box(value=[[(89, 10), (97, 10), (97, 110), (89, 110), (89, 10)]]), polygon=None, raster=None, embedding=None, is_instance=True, implied_task_types=None)]), Prediction(datum=Datum(uid='img4', metadata={'path': 'a/b/c/img4.png'}), annotations=[Annotation(labels=[Label(key='class_label', value='dog', score=0.8), Label(key='class_label', value='cat', sco

## Evaluating Performance

Finally, we'll use our Valor abstractions to evaluate model performance. For more detailed, task-specific examples, see our follow-up notebooks at the links below:

- [Tabular classification](https://github.com/Striveworks/valor/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/valor/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/valor/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)


### Evaluating Detections


In [4]:
eval_objdet = evaluate_detection(groundtruths=detection_gts, predictions=detection_pds)
eval_objdet.metrics

[{'label': {'key': 'class_label', 'value': 'person'},
  'parameters': {'iou': 0.5},
  'value': 1.0,
  'type': 'AP'},
 {'label': {'key': 'class_label', 'value': 'person'},
  'parameters': {'iou': 0.75},
  'value': 1.0,
  'type': 'AP'},
 {'label': {'key': 'class_label', 'value': 'cat'},
  'parameters': {'iou': 0.5},
  'value': 1.0,
  'type': 'AP'},
 {'label': {'key': 'class_label', 'value': 'cat'},
  'parameters': {'iou': 0.75},
  'value': 1.0,
  'type': 'AP'},
 {'label': {'key': 'class_label', 'value': 'dog'},
  'parameters': {'iou': 0.5},
  'value': 1.0,
  'type': 'AP'},
 {'label': {'key': 'class_label', 'value': 'dog'},
  'parameters': {'iou': 0.75},
  'value': 1.0,
  'type': 'AP'},
 {'parameters': {'label_key': 'class_label', 'iou': 0.5},
  'value': 1.0,
  'type': 'mAP'},
 {'parameters': {'label_key': 'class_label', 'iou': 0.75},
  'value': 1.0,
  'type': 'mAP'},
 {'label': {'key': 'class_label', 'value': 'person'},
  'parameters': {'ious': [0.5,
    0.55,
    0.6,
    0.65,
    0.7,

In [5]:
eval_clf = evaluate_classification(groundtruths=classification_gts, predictions=classification_pds)
eval_clf.metrics

[{'label': {'key': 'class_label', 'value': 'cat'},
  'value': 1.0,
  'type': 'Precision'},
 {'label': {'key': 'class_label', 'value': 'cat'},
  'value': 1.0,
  'type': 'Recall'},
 {'label': {'key': 'class_label', 'value': 'cat'}, 'value': 1.0, 'type': 'F1'},
 {'label': {'key': 'class_label', 'value': 'dog'},
  'value': 1.0,
  'type': 'Precision'},
 {'label': {'key': 'class_label', 'value': 'dog'},
  'value': 1.0,
  'type': 'Recall'},
 {'label': {'key': 'class_label', 'value': 'dog'}, 'value': 1.0, 'type': 'F1'},
 {'parameters': {'label_key': 'class_label'},
  'value': 1.0,
  'type': 'Accuracy'},
 {'parameters': {'label_key': 'class_label'}, 'value': 1.0, 'type': 'ROCAUC'}]

## Next Steps

For more examples, we'd recommend reviewing our [other sample notebooks on GitHub](https://github.com/Striveworks/valor/blob/main/examples/). For more detailed explanations of Valor's technical underpinnings, see our [technical concepts guide](technical_concepts.md).
